# Equilibria and Optimal Fitness Functions

This notebook is used to write and test functions for finding the equilibria, based on our model and mangel and clark 1980 something model

In [56]:
%%writefile equilibria_funs.py
import numpy as np
import matplotlib.pyplot as plt
from fitness_funs_non_dim import *
from group_w_pop_funs import *
from scipy.integrate import solve_ivp
from scipy.optimize import root
from local_stability_funs import *
from sim_graph_funs import *




def find_mangel_clark(N1, N2, x_max, **params):
    # mangel and clark predicted that groups should grow until W(x^*) = W(1)
    # don't need A

    # simplest way... iterate and stop when reach x s.t. W(x) < W(1), then return x - 1
    W_of_1 = per_capita_fitness_from_prey_non_dim(1, N1, N2, **params)
    for x in range(2,x_max+1):
        W_of_x = per_capita_fitness_from_prey_non_dim(x, N1, N2, **params)
        if W_of_x < W_of_1:
            return x - 1
    return x # if reach x_max


def iterate_to_eq(initialstate, t_f, params):
    '''
    try to iterate to eq in t_f time steps
    return curr (the final point after iterations), success, timederivatives 
    '''
    out2 = bounded_ivp(initialstate, params, t_f = t_f) 
    T, N1, N2, P, g_of_x_vec, mean_x = out2
    
    # extract results
    traj = [N1,N2,*g_of_x_vec]
    curr = [item[-1] for item in traj]
    [N1,N2,*g_eq] = curr

    # check if at equilibrium
    
    success, timederivatives = check_at_equilibrium2(
        curr[0], curr[1], curr[2:], params
    )

    # get mean experienced
    x_max = params["x_max"]
    P_eq = P[-1]; 
    
    mean_x_eq = mean_group_size_membership(g_eq,x_max,P_eq)

    curr.append(mean_x_eq)

    if not np.isfinite(curr).all():
        success = False
    
    
    return curr, success, timederivatives 
   
def get_equilibrium(params,N1_0,N2_0,g_of_x_vec):#, N1_0 = 0.5, N2_0 = 0.4, p_0 = 20, g_of_x_vec = None):
    '''
    finds the equilibrium s.t. N1, N2 > 0 using root for the population dynamics and group dynamics system
    RETIRED: if not given g_of_x_vec, then just has everyone initially solitary
    
    @returns:
    N1_eq, N2_eq, F_eq, P_eq, mean_x_eq
    '''
    x_max = params['x_max']
    xvec = np.arange(1,x_max+1,1)
    # if not isinstance(g_of_x_vec, np.ndarray):
    #     #print('hi')
    #     x_f = 2 if x_max > 2 else x_max
    #     g_of_x_vec = initiate_f_first_x(p_0, x_f, x_max)
        
    x0 = [N1_0, N2_0, *g_of_x_vec]
    out = root(fun = nullclines_no_P, x0 = x0, 
                                  args = (params))
    return out

def abs_nullclines_no_P(initialstate, params):
    return np.sum(np.abs(nullclines_no_P(initialstate, params)))



def nullclines_no_P(initialstate, params):
    '''
    returns the nullclines for N1, N2, g(1), g(2), ..., g(x_max)
    such that N1, N2 \neq 0
    @inputs
    initialstate = [N1, N2, g(1), ..., g(x_max)], type ndarray
    params = dictionary of params
    '''
    N1 = initialstate[0]
    N2 = initialstate[1]
    g_of_x_vec = initialstate[2:]

    x_max = params['x_max']
    xvec = np.arange(1,x_max+1,1)


    
    

    N1_null, N2_null = N_nullclines(N1, N2, g_of_x_vec, xvec, **params)
    dgdT_vec = group_formation_model_non_dim(0, g_of_x_vec,N1,N2, params) # I put 0 for T
    
    return [N1_null, N2_null, *dgdT_vec]
    
def nullclines_big_prey_extinct(initialstate, params):
    '''
    returns the nullclines for N2,  g(1), g(2), ..., g(x_max)
    where N1 = 0 and N2 > 0
    @inputs
    initialstate = N2, g(1), ..., g(x_max)], type ndarray
    params = dictionary of params
    '''
    
    N2 = initialstate[0]
    g_of_x_vec = initialstate[1:]
    x_max = params['x_max']
    xvec = np.arange(1, x_max+1,1)
    _, N2_null = N_nullclines(0, N2, g_of_x_vec, xvec, **params)
    dgdT_vec = group_formation_model_non_dim(0, g_of_x_vec,0,N2, params) # put 0 for T, N1

    return [N2_null, *dgdT_vec]

def nullclines_small_prey_extinct(initialstate, params):
    '''
    returns the nullclines for N1,  g(1), g(2), ..., g(x_max)
    where N2 = 0 and N1 > 0
    @inputs
    initialstate = [N1, g(1), ..., g(x_max)],
    params = dictionary of params
    '''
    
    N1 = initialstate[0]
    g_of_x_vec = initialstate[1:]
    x_max = params['x_max']
    xvec = np.arange(1, x_max+1,1)
    N1_null, _ = N_nullclines(N1, 0, g_of_x_vec, xvec, **params)
    dgdT_vec = group_formation_model_non_dim(0, g_of_x_vec,N1,0, params) # put 0 for T, N1

    return [N1_null, *dgdT_vec]

def N_nullclines(N1, N2, g_of_x_vec, xvec, η1, η2, A, H1, H2, **params):
    '''
    dN1dT, dN2dT, the change in prey pop size versus time, non-dim'ed, divided by N_i
    @inputs:
    N1, N2 - non-dim'ed pred, big prey, and small prey pop sizes
    g_of_x_vec - array of g(1), g(2), ... , g(x_max)
    params - dic of params: must at least include H1, H2, α1_of_1, α2_of_1, s1, s2,
    '''

    α1 = fun_alpha1(xvec,**params) 
    α2 = fun_alpha2(xvec,**params) 

    # prey nonzero nullclines
    denominator = 1 + H1*α1*N1/xvec + H2*α2*N2/xvec
    Y1_no_N = α1/denominator
    Y2_no_N = α2/denominator

    N1_null = η1 * (1-N1) - A * np.sum(g_of_x_vec * Y1_no_N)
    N2_null = η2 * (1-N2) - A * np.sum(g_of_x_vec * Y2_no_N)
    
    return N1_null, N2_null
def get_equilibrium_prey_i_extinct(params, i, Nj_0 = 0.4, 
                                p_0 = 20, g_of_x_vec = None):
    '''
    finds the equilibrium using root for the population dynamics and group dynamics system
    where N1 = 0
    if not given g_of_x_vec, then just has everyone initially solitary
    
    @returns:
    N1_eq, N2_eq, F_eq, P_eq, mean_x_eq
    '''
    x_max = params['x_max']
    xvec = np.arange(1,x_max+1,1)
    if not isinstance(g_of_x_vec, np.ndarray):
        #print('hi')
        x_f = 2 if x_max > 2 else x_max
        g_of_x_vec = initiate_f_first_x(p_0, x_f, x_max)
        
    x0 = [Nj_0, *g_of_x_vec]
    if i == 1:    
        out = root(fun = nullclines_big_prey_extinct, x0 = x0, 
                                  args = (params))
    elif i == 2:
        out = root(fun = nullclines_small_prey_extinct, x0 = x0, args = (params))
    return out  


def check_at_equilibrium2(N1,N2,g_of_x_vec, params):
    # check not negative
    curr = [N1, N2, *g_of_x_vec]
    condition_failed_1 = np.any(
        np.array(curr)<0
    )
    deriv_vec = full_model(
        0, curr, True, params
    )
    condition_failed_2 = np.any(np.abs(deriv_vec)>1e-8)
    if np.any([condition_failed_1, condition_failed_2]):#, condition_failed_3]):
        success = False
    else:
        success = True
    return success, deriv_vec
    # check derivative is zero
    # check sum x*g(x) = p
def get_results_eq(out, x_max, tol = 1e-8, which_prey_extinct = -1):
    '''
    Extracts the state variables at the equilibrium, calculates 
    mean experienced group size, and checks that the equilibrium 
    is valid (within the state variable domains)

    @ returns: P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success
    '''
    xvec = np.arange(1,x_max+1,1)
    if which_prey_extinct == -1:
        g_eq = out.x[2:]
        N1_eq = out.x[0]
        N2_eq = out.x[1]
    else:
        g_eq = out.x[1:]
        Nj_eq = out.x[0]
        Ni_eq = 0
        N1_eq = Ni_eq if which_prey_extinct == 1 else Nj_eq
        N2_eq = Ni_eq if which_prey_extinct == 2 else Nj_eq
    P_eq = np.sum(xvec*g_eq); 
    
    mean_x_eq = mean_group_size_membership(g_eq,x_max,P_eq)
    if mean_x_eq < 1:
        mean_x_eq = 1
    # check not negative
    condition_failed_1 = np.any(np.array([P_eq, N1_eq, N2_eq, *g_eq, mean_x_eq])<0)
    # check root reached the end
    condition_failed_2 = out.success == False
    # check sum x*g(x) = p
    #condition_failed_3 = np.abs(np.sum(np.arange(1,x_max+1,1)*g_eq) - P_eq) > tol
    
    if np.any([condition_failed_1, condition_failed_2]):#, condition_failed_3]):
        success = False
        return np.nan, np.nan, np.nan, np.nan, np.nan, success
    success = True
    return P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success

def initiate_g_first_x(x_f, x_max):
    
    g0 = np.zeros(x_max) + 1e-4
    g0[0:x_f] = 1
    return g0

def iterate_and_solve_equilibrium(params, t_f = 1000, tol = 1e-8):
    '''
    iterates from a standard start point that tends to work
    then uses root to find equilibrium

    @returns
    [N1,N2,*g, mean_x], success (BOOL for whether at equilibrium), 
    and a vector of the time derivatives
    '''
    x_max = params['x_max']
    x_f = 3 if x_max >=3 else 2
    y0 = [0.71, 0.7, *initiate_g_first_x(3, x_max)]
    out2 = bounded_ivp(y0, params, t_f = t_f) 
    T, N1, N2, P, g_of_x_vec, mean_x = out2

    # extract new starting point
    traj = [N1,N2,*g_of_x_vec]
    curr = [item[-1] for item in traj]
    print(curr)

    out = get_equilibrium(params, N1_0 = curr[0], N2_0 = curr[1], 
                          g_of_x_vec = curr[2:])
    P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success =get_results_eq(out,x_max)

    # to be successful, sum x*g = P
    # sum_x_g = np.sum(np.arange(1,x_max+1,1)*g_eq)
    # success = success and (np.abs(sum_x_g - P_eq )< tol)
    
    return P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success
    
######################################################################

# check and update functions below, if still needed

    
def get_equilibria_vary_param(paramvec, paramkey, **params):
    '''
    Get a list of equilibrium values corresponding to the parameters
    '''


    x_max = params['x_max']
    xvec = np.arange(1,x_max+1,1)

    # set up empty vectors
    meanxvec = np.zeros(len(paramvec))
    gxvecs  = np.zeros((len(paramvec), x_max))
    Pvec = meanxvec.copy()
    N1vec = meanxvec.copy()
    N2vec = meanxvec.copy()
    success_vec = meanxvec.copy()
    stability_vec = meanxvec.copy()
    
    for i, param in enumerate(paramvec):
        params = params.copy()
        params[paramkey] = param

        # try to iterate a little and then use root to solve for equilibrium
        out_eq = iterate_and_solve_equilibrium(params, t_f = 5)
        P, N1, N2, g, mean_x, success = out_eq
        
        if success==False:
            
            # try to get to equilibrium in just 200 steps #
            
            t_f = 500
            initialstate = [0.5,0.4, 20, *np.zeros(x_max-1)]
            finalpoint, success, mean_x, _, _ = iterate_to_eq(initialstate, t_f,
                                                                         params)
            [P,N1,N2,*g] = finalpoint

            # if that doesn't work, try solving from here
            if success == False:
                out = get_equilibrium(params, N1_0 = N1, N2_0 = N2, 
                          g_of_x_vec = g)
                P, N1, N2, g, mean_x, success =get_results_eq(out,x_max)
            # if that doesn't work, now do another 2000 steps
            if success == False:
                out = iterate_to_eq(finalpoint[1:], 5000,params)   
                finalpoint, success, mean_x, _, _ = out
            
                [P,N1,N2,*g] = finalpoint
            if success == False:
                out = get_equilibrium(params, N1_0 = N1, N2_0 = N2, 
                          g_of_x_vec = g)
                P, N1, N2, g, mean_x, success =get_results_eq(out,x_max)
            
        success_vec[i] = success
        
        gxvecs[i,:] = g
        Pvec[i] = P
        N1vec[i] = N1
        N2vec[i] = N2
        meanxvec[i] = mean_x


        # check stability
        try:
            if np.any(np.isnan(np.array([P,N1,N2,*g]))):
                stability_vec[i] = np.nan
        except TypeError:
            stability_vec[i] = np.nan
        else:
            J = fun_Jac(N1,N2,np.array(g),**params)
            stability = classify_stability(J)
            if stability == "Stable (attractive)":
                stability_vec[i] = 1
            elif stability == "Unstable":
                stability_vec[i] = -1
            else:
                stability_vec[i] = 0
        
    return Pvec, N1vec, N2vec, gxvecs,meanxvec,success_vec, stability_vec
 

''' 
retired i think
def check_at_equilibrium(final_distribution, P, N1, N2,**params):
    
    # check dg(x)/dT \approx 0
    # @ returns: array dgdT_, and 1 if at equilibrium or 0 if not
    
    T = 1 # this doesn't matter
    dgdT_ = group_formation_model_non_dim(T, final_distribution,N1,N2, params)
    not_at_equilibrium = np.abs(dgdT_) > 1e-8
    if sum(not_at_equilibrium) > 0: # at least one dg(x)/dt is not zero
        return dgdT_, 0 # 0 means not at equilibrium
    else:
        return dgdT_, 1 # 1 means not at equilibrium

'''

Overwriting equilibria_funs.py


# Check iterate_to_eq

In [50]:
t_f = 1100
weird_params = {'η1': np.float64(0.5), 'η2': 0.5, 'A': 0.5, 'β1': 8, 'β2': 1, 'H1': 0, 'H2': 0, 'α1_of_1': 0.05, 'α2_of_1': 0.95, 's1': 2, 's2': 2, 'α2_fun_type': 'constant', 
 'x_max': 5, 'd': 10, 'Tx': 0.01, 'pop_process': True}
initialstate = np.array([.7,.7,*initiate_g_first_x(3,weird_params["x_max"])])


In [51]:
out2 = bounded_ivp(initialstate, weird_params, t_f = t_f) 

In [52]:
T, N1, N2, P, g_of_x_vec, mean_x = out2
traj = [N1,N2,*g_of_x_vec]
curr = [item[-2] for item in traj]


In [53]:
curr

[np.float64(2.2704568997372414e-244),
 np.float64(2.4817798352037342e-244),
 np.float64(0.04362839394769),
 np.float64(0.12919990861272634),
 np.float64(1.9416920119364407),
 np.float64(1.8911354264097888),
 np.float64(0.16192956025631405)]

In [54]:
coexist_eq, success, _ = iterate_to_eq(initialstate, t_f, weird_params)
coexist_eq = np.array(coexist_eq)

In [55]:
success

False

# Check that get_equilibrium works:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fitness_funs_non_dim import *
from group_w_pop_funs import *
from scipy.integrate import solve_ivp
from scipy.optimize import root
from local_stability_funs import *
from equilibria_funs import *

In [2]:
x_max = 5
params_base = dict(η1 = 0.2, η2 = 0.5, A = 0.5, β1 = 8, β2 = 1, H1=8, H2=1, 
                  α1_of_1=0.05, α2_of_1=0.95, 
                  s1=2, s2=2, α2_fun_type = 'constant',
                  x_max = x_max, d = 10,
                 Tx = .01, pop_process = True)
g_of_x_vec = [1,0,0,0,0]
xvec = np.array([1,2,3,4,5])

In [3]:
[N1_0,N2_0,*g_of_x_vec] = [0.62637957, 0.69268158, 0.23320018, 0.15116554, 
                           0.20782921, 0.08853843, 0.01187339]

p_0 = sum(np.arange(1,6,1)*np.array(g_of_x_vec))

In [4]:
out = get_equilibrium(params_base.copy(), N1_0 , N2_0, p_0, g_of_x_vec)
print(out.x)
out.success

[0.6340475  0.72130594 0.23320018 0.15116554 0.20782921 0.08853843
 0.01187339]


True

In [32]:
full_model?

Signature: full_model(T, initialstate, arg, params)
Docstring:
removed P!
gets the time derivatives for N1, N2, g(1), g(2), ..., g(xm)
@inputs
T is just used by fsolve, not needed
intiialstate = [N1,N2,*g_of_x]
arg is a dummy because fsolve gets weird if there is only 1 arg?
params is dictionary of params
@ returns [dN1dT, dN2dT, *dgdT_vec]
File:      ~/Documents/CH_GroupFormation/CH_code/Functions/group_w_pop_funs.py
Type:      function

In [34]:
perturbed_pt = out.x + .01
out2 = solve_ivp(full_model, [0, 5000], perturbed_pt, 
                         method = "LSODA", args = (True, params_base.copy()))

In [37]:
out2.y[:,-1]

array([0.6340475 , 0.72130594, 0.23320018, 0.15116554, 0.20782921,
       0.08853843, 0.01187339])

# Iterate and solve equilibrium

In [23]:

x_max = 5
params_base = dict(η1 = 0.2, η2 = 0.5, A = 0.5, β1 = 12, β2 = 1, H1=12, 
                   H2=1, 
                  α1_of_1=0.05, α2_of_1=0.95, 
                  s1=2, s2=2, α2_fun_type = 'constant',
                  x_max = x_max, d = 10,
                 Tx = .01, pop_process = True)

In [27]:
initialstate = [.7,.7,*initiate_g_first_x(3,5)]
t_f = 1500
params = params_base.copy()
coexist_eq, success, _ = iterate_to_eq(initialstate, t_f, params)

In [17]:
def initiate_g_first_x(x_f, x_max):
    
    g0 = np.zeros(x_max) + 1e-4
    g0[0:x_f] = 1
    return g0

def iterate_and_solve_equilibrium(params, t_f = 1000, tol = 1e-8):
    '''
    iterates from a standard start point that tends to work
    then uses root to find equilibrium

    @returns
    [N1,N2,*g, mean_x], success (BOOL for whether at equilibrium), 
    and a vector of the time derivatives
    '''
    x_max = params['x_max']
    x_f = 3 if x_max >=3 else 2
    y0 = [0.71, 0.7, *initiate_g_first_x(3, x_max)]
    out2 = bounded_ivp(y0, params, t_f = t_f) 
    T, N1, N2, P, g_of_x_vec, mean_x = out2

    # extract new starting point
    traj = [N1,N2,*g_of_x_vec]
    curr = [item[-1] for item in traj]
    print(curr)

    out = get_equilibrium(params, N1_0 = curr[0], N2_0 = curr[1], 
                          g_of_x_vec = curr[2:])
    P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success =get_results_eq(out,x_max)

    # to be successful, sum x*g = P
    # sum_x_g = np.sum(np.arange(1,x_max+1,1)*g_eq)
    # success = success and (np.abs(sum_x_g - P_eq )< tol)
    
    return P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success

In [18]:
iterate_and_solve_equilibrium(params_base)

[np.float64(0.4267059462047271), np.float64(0.6443679343578119), np.float64(0.2098708248826281), np.float64(0.17192597060135076), np.float64(0.30840995616937295), np.float64(0.1739846341313669), np.float64(0.030953298511645355)]


(nan, nan, nan, nan, nan, False)

In [ ]:
P_eq, N1_eq, N2_eq, g_eq, mean_x_eq, success